In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.datasets import load_iris
from scipy.stats import spearmanr
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import time

READING THE PRE-PROCESSED DATA

In [ ]:
# load processed data
ef = pd.read_csv('kdd_processed_bin.csv')

CREATING AND TRAINING A THREE LAYERED LSTM MODEL (ACTIVATION AS SIGMOID)

In [8]:
# train an lstm model on the data
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from sklearn import model_selection
from sklearn import metrics
X = ef.drop('class', axis=1)
Y = ef['class']

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

# convert Y to int
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# reshape the data for LSTM
X_train = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

# convert all to numpy arrays
X_train = np.asarray(X_train).astype(np.float32)
X_test = np.asarray(X_test).astype(np.float32)


# define the model
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(64))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
start = time.time()
# train the model
history = model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test), shuffle=True)
end = time.time()
print("Time taken to train the model: ", end-start)

Epoch 1/10
3150/3150 [==============================] - 8s 2ms/step - loss: 0.0703 - accuracy: 0.9732 - val_loss: 0.0409 - val_accuracy: 0.9846
Epoch 2/10
3150/3150 [==============================] - 7s 2ms/step - loss: 0.0329 - accuracy: 0.9879 - val_loss: 0.0348 - val_accuracy: 0.9868
Epoch 3/10
3150/3150 [==============================] - 6s 2ms/step - loss: 0.0275 - accuracy: 0.9898 - val_loss: 0.0285 - val_accuracy: 0.9909
Epoch 4/10
3150/3150 [==============================] - 6s 2ms/step - loss: 0.0249 - accuracy: 0.9909 - val_loss: 0.0310 - val_accuracy: 0.9889
Epoch 5/10
3150/3150 [==============================] - 6s 2ms/step - loss: 0.0226 - accuracy: 0.9919 - val_loss: 0.0301 - val_accuracy: 0.9892
Epoch 6/10
3150/3150 [==============================] - 6s 2ms/step - loss: 0.0213 - accuracy: 0.9925 - val_loss: 0.0250 - val_accuracy: 0.9923
Epoch 7/10
3150/3150 [==============================] - 6s 2ms/step - loss: 0.0203 - accuracy: 0.9930 - val_loss: 0.0234 - val_accuracy:

EVALUATING THE MODEL

In [10]:
start_time = time.time()
y_pred = model.predict(X_test)
end_time = time.time()
y_pred = (y_pred > 0.5)
print('Classification Report')
print(metrics.classification_report(Y_test, y_pred))
print('Accuracy:')
print(metrics.accuracy_score(Y_test, y_pred))

print('Prediction time:', end_time-start_time)

788/788 [==============================] - 1s 587us/step
Classification Report
              precision    recall  f1-score   support

           0       0.99      0.99      0.99     13422
           1       0.99      0.99      0.99     11773

    accuracy                           0.99     25195
   macro avg       0.99      0.99      0.99     25195
weighted avg       0.99      0.99      0.99     25195

Accuracy:
0.9926969636832705
Prediction time: 0.7456660270690918


In [12]:
# print latency for model prediction
start = time.time()
y_pred = model.predict(X_test)
end = time.time()
print('Average Latency for model prediction:(ms)')
lat = ((end-start)/len(y_pred))
# convert lat to ms
lat = lat*1000
print(lat)

788/788 [==============================] - 0s 596us/step
Average Latency for model prediction:(ms)
0.02281249906127712


In [13]:
# save the model
model.save('kdd_binary.h5')

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Cross Validation of the model

In [26]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

def get_model():
    model = Sequential()
    model.add(LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(64, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(64))
    model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(build_fn=get_model, epochs=10, batch_size=32, verbose=0)
kfold = KFold(n_splits=5, shuffle=True)
X = ef.drop('class', axis=1)
Y = ef['class']

# reshape the data for LSTM
X_fit = X.values.reshape((X.shape[0], 1, X.shape[1]))
X_fit = np.asarray(X_fit).astype(np.float32)

results = cross_val_score(model, X_fit, Y, cv=kfold)
print("Accuracy: %.2f%%" % (results.mean()*100.0))

C:\Users\Ameya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\Ameya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\Ameya\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initia

Accuracy: 98.71%


In [ ]:
# save mode
model.save('lstm_bin.h5')